In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import pandas as pd
import numpy as np
train=pd.read_csv('../Allstate/data/train.csv')
test=pd.read_csv('../Allstate/data/test.csv')
loss=train['loss']
ids=train['id']
test_id=test['id']
train.drop(['id','loss'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)
n=train.shape[0]
data=pd.concat([train,test],axis=0,ignore_index=True)

OSError: File b'../Allstate/data/train.csv' does not exist

In [ ]:

label=LabelEncoder()
enc=OneHotEncoder()
cols=train.columns.tolist()
cat_cols=data.filter(regex='cat').columns.tolist()
for item in cat_cols:
    temp=list(set(data[item].values))
    temp.sort()
    if len(temp)==2:
        data.loc[data[item]==temp[0],item]=0
        data.loc[data[item]==temp[1],item]=1
    if len(temp)>2 :
        data[item]=label.fit_transform(data[item])
        new=pd.DataFrame(enc.fit_transform(data[item].reshape(-1,1)).toarray(),dtype=np.int8)
        col_names=[]
        for i in range(new.shape[1]):
            col_names.append(item+"_"+str(i)) 
        new.columns=col_names
        data.drop(item,inplace=True,axis=1)
        data=pd.concat([data,new],axis=1)

In [ ]:
train=data.iloc[:n,]
test=data.iloc[n:,]
#train.to_csv('../Allstate/data/train_ohe.csv',index=False)
#test.to_csv('../Allstate/data/test_ohe.csv',index=False)

In [ ]:
train=data.iloc[:n,]
test=data.iloc[n:,]
train.to_csv('./kaggle/Allstate/data/train_ohe.csv',index=False)
test.to_csv('./kaggle/Allstate/data/test_ohe.csv',index=False)

In [ ]:

#xgbmodel needs optimize

import xgboost as xgb
dtrain = xgb.DMatrix(train.loc[1000:,].as_matrix(),label=loss.loc[1000:,].as_matrix())
dtest = xgb.DMatrix(train.loc[:1000,].as_matrix(),label=loss.loc[:1000,].as_matrix())
watchlist  = [(dtrain,'train'),(dtest,'test')]
param = {'max_depth':6, 
         'eta':0.07, 
         'silent':False, 
         'objective':'reg:linear', 
         'verbose_eval':True, 
         'eval_metric' : 'mae', 
         'colsample_bytree': 0.7,
         'subsample': 0.7,
         'num_parallel_tree': 1,
         'min_child_weight': 1}
num_round = 250
bst = xgb.train(param, dtrain, num_round, watchlist)   

In [16]:
train=pd.read_csv('../Allstate/data/train_ohe.csv')
train.drop(['loss'],inplace=True,axis=1)
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
rf_tree=200
rf_depth=10
#1223 n=200, min_split=2, min_leaf=1
rf=RandomForestRegressor(n_estimators=rf_tree, criterion='mse', max_depth=None,
                         min_samples_split=2, min_samples_leaf=1, max_features=None, 
                         max_leaf_nodes=None,   
                         oob_score=True, n_jobs=4, random_state=0, verbose=1)

#1260 n=200, min_split=2, min_leaf=1
from sklearn.ensemble import ExtraTreesRegressor
et_tree=200
et=ExtraTreesRegressor(n_estimators=et_tree,max_depth=None,min_samples_split=2, min_samples_leaf=1,max_features=None,verbose=1,n_jobs=4)

a=[10,20,30,40,50,60,70,80,90,100]
X_train, X_test, y_train, y_test = train_test_split(train, loss, test_size=0.1, random_state=42)
trees=np.array([])
MAE=np.array([])
for i in a:
    rf=RandomForestRegressor(n_estimators=i, criterion='mse', max_depth=None,
                         min_samples_split=2, min_samples_leaf=1, max_features=None, 
                         max_leaf_nodes=None,   
                         oob_score=True, n_jobs=4, random_state=0, verbose=1)
    rf.fit(X_train,y_train)
    pred=rf.predict(X_test)
    mae=mean_absolute_error(y_test,pred)
    trees=np.append(trees,[i])
    MAE=np.append(MAE,[mae])
    plt.scatter(trees, MAE, alpha=0.5)
    plt.show()

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  1.6min finished
C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:723: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  2.8min finished
C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:723: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, loss, test_size=0.1, random_state=42)
rf.fit(X_train,y_train)
a=rf.predict(X_test)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
X_train, X_test, y_train, y_test = train_test_split(train, loss, test_size=0.2, random_state=42)

et.fit(X_train,y_train)
#rf.fit(X_train,y_train)

#rf_pred=rf.predict(X_test)
et_pred=et.predict(X_test)

#print(mean_absolute_error(y_test,rf_pred))
print(mean_absolute_error(y_test,et_pred))


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  9.8min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 43.4min
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed: 45.1min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    5.8s finished


1259.55956341


In [ ]:
b=rf.predict(X_test)
a=et.predict(X_test)
mean_absolute_error(a,y_test)

In [ ]:
test=pd.read_csv('../Allstate/data/test_ohe.csv')

In [ ]:
train.shape

In [ ]:
n

In [ ]:
data

In [3]:
out=pd.DataFrame(columns={'trees','MAE'})

In [5]:
out.append([1,2])

,MAE,trees,0
0,NaN,NaN,1.0
1,NaN,NaN,2.0


In [6]:
out.append

<bound method DataFrame.append of Empty DataFrame
Columns: [MAE, trees]
Index: []>

In [7]:
df = pd.DataFrame([[i, mean_absolute_error(y_test,pred)]], columns=list('AB'))

,MAE,trees


In [8]:
a=np.array([])

In [9]:
a

array([], dtype=float64)

In [11]:
a=np.append(a,[1])

In [12]:
a

array([ 1.])

In [13]:
a=np.append(a,[3])

In [18]:
trees

array([ 10.,  20.])

In [19]:
import matplotlib.pyplot as plt

plt.scatter(trees, MAE, alpha=0.5)
plt.show()